# ab13dd Example

Johannes Kaisinger, 26 July 2023

In [2]:
import numpy as np
import scipy.linalg as linalg
import slycot

In [3]:
A = np.array([
    [-1, 10],
    [0, -1]
])

B = np.array([
    [0],
    [1]])

C = np.array([
    [1, 0]])
D = np.zeros((1,1))

n, m = B.shape
p, _ = C.shape

## Slycot H-infinity Norm

In [4]:
out = slycot.ab13dd('C', 'I', 'N', 'D', n, m, p, A, np.eye(n), B, C, D)
norm_sylcot, _ = out
print(norm_sylcot)

10.0


## Bisection algorithm H-infinity Norm

In [5]:
def H_inf(A,B,C,D,gam_l,gam_h,emin):
    """naive implementation of bisection algorithm for H-infinity norm

    Args:
        A (_type_): _description_
        B (_type_): _description_
        C (_type_): _description_
        D (_type_): _description_
        gam_l (_type_): _description_
        gam_h (_type_): _description_
        emin (_type_): _description_

    Returns:
        _type_: _description_
    """
    gam_last_stable = None
    while (gam_h - gam_l) > emin:
        gam = (gam_l+gam_h)/2
        R = gam**2*np.eye(1)-D.T@D
        R_inv = linalg.inv(R)
        Mgam = np.vstack((
        np.hstack((A+B@R_inv@D.T@C, B@R_inv@B.T)),
        np.hstack((-C.T@(np.eye(1)+D@R_inv@D.T)@C, -(A+B@R_inv@D.T@C).T))))
        d = linalg.eigvals(Mgam)
        if np.any(np.imag(d)):
            gam_l = gam
        else:
            gam_h = gam
            gam_last_stable = gam
    return gam_last_stable

In [6]:
norm_bi = H_inf(A,B,C,D,0.001,100,1e-10)
print(norm_bi)

10.000000000000638


## Compare

In [7]:
# compare results
print(norm_sylcot)
print(norm_bi)
np.allclose(norm_sylcot,norm_bi)


10.0
10.000000000000638


True